# Obtenção e organização dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/excertos_filtrados500.csv', sep = '|')[['DESCR_AREA', 'filtrado']]
df.head()

,DESCR_AREA,filtrado
0,Responsabilidade,voto cuidar auto tomada conta especial instaur...
1,Finanças Públicas,voto cuidar auto solicitação congresso naciona...
2,Responsabilidade,relatório tratar embargo declaração opor exemp...
3,Direito Processual,voto relação outro processo judiciais tratar r...
4,Pessoal,voto relativo ato envolver senhor caber rememo...


In [2]:
df.shape

(13285, 2)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['DESCR_AREA']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['DESCR_AREA'])
lbArea.classes_, y.shape

(array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
        'Desestatização', 'Direito Processual', 'Finanças Públicas',
        'Gestão Administrativa', 'Licitação', 'Pessoal',
        'Responsabilidade'], dtype='<U23'), (13285, 10))

# Pré-processamento

In [4]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

print('\tCarregamento do modelo de acordaos...')
modelo = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

	Carregamento do modelo de acordaos...


/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
import numpy as np
from keras.preprocessing.text import Tokenizer

limite_texto = 500
dim_vetor = 100

print('\tTokenizacao e montagem de sequencias...')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['filtrado'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['filtrado'])

print('\tMontagem da matriz de embeddings...')
embedding_matrix = np.zeros((vocabulario, dim_vetor))
for word, i in tokenizer.word_index.items():
    if word in modelo.wv:
        embedding_matrix[i] = modelo.wv[word]

	Tokenizacao e montagem de sequencias...
	Montagem da matriz de embeddings...


# Treinamentos com Cross-validation

### Modelos a serem comparados:

* Teste 1 - Excertos filtrados (500) com rede recorrente
* Teste 2 - Excertos filtrados (500) com rede convolucional

In [8]:
colunas_scores = list(lbArea.classes_)
colunas_scores.extend(['accuracy', 'macro avg', 'weighted avg'])
alternativas = ['rede recorrente sobre texto filtrado', 'rede convolucional sobre texto filtrado']

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D, Flatten, GRU
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences

df_medias = pd.DataFrame()
for alt in range(0, 2):
    str_alt = str(alt)
    fold = 0
    df_scores = pd.DataFrame()

    print('\n\nAlternativa', alt+1, '-', alternativas[alt])
    
    for train_index, val_index in KFold(n_splits=10, random_state=42, shuffle=True).split(df):
        str_fold = 'Fold ' + str(fold)
        print()
        print()
        print('Alternativa ' + alternativas[alt], '-', str_fold + ':')
        print('\tDefinicao de valores de entrada e saida da rede...')

        df_train = df.loc[train_index]
        df_val = df.loc[val_index]

            
        sequences_train = tokenizer.texts_to_sequences(df_train['filtrado'])
        sequences_val = tokenizer.texts_to_sequences(df_val['filtrado'])

        x_train = pad_sequences(sequences_train, maxlen=limite_texto)
        x_val = pad_sequences(sequences_val, maxlen=limite_texto)

        y_train = lbArea.transform(df_train['DESCR_AREA'])
        y_val = lbArea.transform(df_val['DESCR_AREA'])

        print('\tTreinamento da rede...')
        model = Sequential()
        model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix]))
        
        
        if alt == 0:
            model.add(GRU(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
            model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
        else:
            model.add(Conv1D(64, 7, activation='relu'))
            model.add(MaxPooling1D(5))
            model.add(Conv1D(32, 7, activation='relu'))
            model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
            
        model.add(Dense(y.shape[1], activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

        checkpoint_filename = 'weights' + str_alt + '-' + str(fold) + '.hdf5'
        checkpointer = ModelCheckpoint(filepath=checkpoint_filename, monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
        model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val), verbose=1, shuffle=False, callbacks=[checkpointer])

        print('\n\tAvaliacao do melhor modelo e registro dos scores...')
        model.load_weights(checkpoint_filename)
        y_val_pred = model.predict_classes(x_val, verbose=1)
        y_val_i = [list(x).index(1) for x in y_val]
        report = classification_report(y_val_i, y_val_pred, target_names=lbArea.classes_, output_dict = True)
        for col in colunas_scores:
            if col == 'accuracy':
                f = report[col]
            else:
                f = report[col]['f1-score']
            df_scores.loc[str_fold,col] = f
        fold += 1
    df_medias[alternativas[alt] + ' mean'] = df_scores.mean()
    df_medias[alternativas[alt] + ' std'] = df_scores.std()
df_medias.T



Alternativa 1 - rede recorrente sobre texto filtrado


Alternativa rede recorrente sobre texto filtrado - Fold 0:
	Definicao de valores de entrada e saida da rede...


W0321 16:14:39.879317 140033094072128 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0321 16:14:39.892505 140033094072128 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0321 16:14:39.894325 140033094072128 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0321 16:14:39.902577 140033094072128 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

	Treinamento da rede...


W0321 16:14:40.806172 140033094072128 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0321 16:14:41.203860 140033094072128 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0321 16:14:41.302528 140033094072128 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0,

Train on 11956 samples, validate on 1329 samples
Epoch 1/20
11956/11956 [==============================] - 345s 29ms/step - loss: 1.1249 - categorical_accuracy: 0.6323 - val_loss: 1.0578 - val_categorical_accuracy: 0.6305

Epoch 00001: val_categorical_accuracy improved from -inf to 0.63055, saving model to weights0-0.hdf5
Epoch 2/20
11956/11956 [==============================] - 342s 29ms/step - loss: 0.7514 - categorical_accuracy: 0.7498 - val_loss: 0.7501 - val_categorical_accuracy: 0.7585

Epoch 00002: val_categorical_accuracy improved from 0.63055 to 0.75847, saving model to weights0-0.hdf5
Epoch 3/20
11956/11956 [==============================] - 342s 29ms/step - loss: 0.6082 - categorical_accuracy: 0.7996 - val_loss: 0.6560 - val_categorical_accuracy: 0.7825

Epoch 00003: val_categorical_accuracy improved from 0.75847 to 0.78254, saving model to weights0-0.hdf5
Epoch 4/20
11956/11956 [==============================] - 343s 29ms/step - loss: 0.5282 - categorical_accuracy: 0.8265 -

...

11956/11956 [==============================] - 35s 3ms/step - loss: 0.1347 - categorical_accuracy: 0.9556 - val_loss: 1.0672 - val_categorical_accuracy: 0.7675

Epoch 00013: val_categorical_accuracy did not improve from 0.77728
Epoch 14/20
11956/11956 [==============================] - 36s 3ms/step - loss: 0.1282 - categorical_accuracy: 0.9569 - val_loss: 1.0931 - val_categorical_accuracy: 0.7570

Epoch 00014: val_categorical_accuracy did not improve from 0.77728
Epoch 15/20
11956/11956 [==============================] - 36s 3ms/step - loss: 0.1124 - categorical_accuracy: 0.9626 - val_loss: 1.1400 - val_categorical_accuracy: 0.7698

Epoch 00015: val_categorical_accuracy did not improve from 0.77728
Epoch 16/20
11956/11956 [==============================] - 36s 3ms/step - loss: 0.0953 - categorical_accuracy: 0.9680 - val_loss: 1.1842 - val_categorical_accuracy: 0.7472

Epoch 00016: val_categorical_accuracy did not improve from 0.77728
Epoch 17/20
11956/11956 [===========================

11957/11957 [==============================] - 35s 3ms/step - loss: 0.8150 - categorical_accuracy: 0.7310 - val_loss: 0.8930 - val_categorical_accuracy: 0.6837

Epoch 00002: val_categorical_accuracy improved from 0.61521 to 0.68373, saving model to weights1-6.hdf5
Epoch 3/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.6785 - categorical_accuracy: 0.7768 - val_loss: 0.7797 - val_categorical_accuracy: 0.7372

Epoch 00003: val_categorical_accuracy improved from 0.68373 to 0.73720, saving model to weights1-6.hdf5
Epoch 4/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.5742 - categorical_accuracy: 0.8121 - val_loss: 0.7519 - val_categorical_accuracy: 0.7485

Epoch 00004: val_categorical_accuracy improved from 0.73720 to 0.74849, saving model to weights1-6.hdf5
Epoch 5/20
11957/11957 [==============================] - 36s 3ms/step - loss: 0.4856 - categorical_accuracy: 0.8374 - val_loss: 0.7303 - val_categorical_accuracy: 0.7643

Epoch 00005

11957/11957 [==============================] - 35s 3ms/step - loss: 0.1360 - categorical_accuracy: 0.9551 - val_loss: 1.1285 - val_categorical_accuracy: 0.7620

Epoch 00013: val_categorical_accuracy did not improve from 0.77937
Epoch 14/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.1302 - categorical_accuracy: 0.9557 - val_loss: 1.1363 - val_categorical_accuracy: 0.7666

Epoch 00014: val_categorical_accuracy did not improve from 0.77937
Epoch 15/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.1102 - categorical_accuracy: 0.9654 - val_loss: 1.1859 - val_categorical_accuracy: 0.7590

Epoch 00015: val_categorical_accuracy did not improve from 0.77937
Epoch 16/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.1083 - categorical_accuracy: 0.9627 - val_loss: 1.1479 - val_categorical_accuracy: 0.7560

Epoch 00016: val_categorical_accuracy did not improve from 0.77937
Epoch 17/20
11957/11957 [===========================


Epoch 00003: val_categorical_accuracy improved from 0.69880 to 0.73870, saving model to weights1-9.hdf5
Epoch 4/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.5816 - categorical_accuracy: 0.8073 - val_loss: 0.7644 - val_categorical_accuracy: 0.7598

Epoch 00004: val_categorical_accuracy improved from 0.73870 to 0.75979, saving model to weights1-9.hdf5
Epoch 5/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.4945 - categorical_accuracy: 0.8373 - val_loss: 0.7269 - val_categorical_accuracy: 0.7688

Epoch 00005: val_categorical_accuracy improved from 0.75979 to 0.76883, saving model to weights1-9.hdf5
Epoch 6/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.4240 - categorical_accuracy: 0.8565 - val_loss: 0.8358 - val_categorical_accuracy: 0.7560

Epoch 00006: val_categorical_accuracy did not improve from 0.76883
Epoch 7/20
11957/11957 [==============================] - 35s 3ms/step - loss: 0.3523 - categorical_accu

,Competência do TCU,Contrato Administrativo,Convênio,Desestatização,Direito Processual,Finanças Públicas,Gestão Administrativa,Licitação,Pessoal,Responsabilidade,accuracy,macro avg,weighted avg
rede recorrente sobre texto filtrado mean,0.698804,0.718493,0.664458,0.752893,0.822335,0.736029,0.687325,0.878605,0.966660,0.791017,0.839744,0.771662,0.838214
rede recorrente sobre texto filtrado std,0.039043,0.038170,0.069102,0.151937,0.024311,0.038413,0.068846,0.014925,0.006864,0.015692,0.012853,0.023767,0.012728
rede convolucional sobre texto filtrado mean,0.552467,0.570548,0.532370,0.489234,0.760240,0.611246,0.547968,0.837406,0.946244,0.727665,0.776667,0.657539,0.772343
rede convolucional sobre texto filtrado std,0.038895,0.049191,0.088349,0.101863,0.024528,0.055860,0.102910,0.020444,0.011575,0.023870,0.017096,0.028832,0.017457


In [10]:
df_medias.T.to_csv('scores_excertos_filtrados.csv', encoding = 'Latin1')